# Example: convert ModelE3 SCM output to DEPHY format

Code to read ModelE3  output files and write to DEPHY format (NetCDF)

Contributed by Ann Fridlind from NASA/GISS

## Import libraries

In [40]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import netCDF4
import datetime as dt
from netCDF4 import Dataset

## Specify directory locations

If on the ARM JupyterHub, it is recommended to create and specify a local directory that is outside of the COMBLE-MIP repository to upload raw model output files in your model's format. 

Processed domain-mean outputs are invited for commit to the GitHub repository on a user-specified branch under /comble-mip/output_scm/YOUR_MODEL_NAME/sandbox/YOUR_OUTPUT_FILE_NAME. These can be committed and removed at any time.

If you are able to make a run without ice, it is requested to append 'noice' to YOUR_OUTPUT_FILE_NAME, so that it can readily be automatically compared with the baseline and other liquid-only runs.

In [41]:
# specify input and output file names: versions of ModelE3 with and without ice

# Phys
my_input_suffix = 'COMBLE_scm_in.nc'
my_prog_suffix = 'progvar.nc'
my_diag_suffix = 'diagvar.nc'
my_diag2_suffix = 'diagvar2.nc'
my_merged_suffix = 'aoscm.nc'

# Phys without ice
# my_input_suffix = 'entppe_noice.nc'
my_output_suffix = 'AOSCM-Phys_FixN_def_z0.nc'



# specify local source directories (with subdirectories for spin up over ice and restart over water)
# my_input_dir = '/data/home/fridlind/modelE3/'
my_input_dir = '~/AOSCM_Karalis/fixed_z0/'
dsprog=xr.open_dataset(my_input_dir + my_prog_suffix,decode_times=False)
dsdiag=xr.open_dataset(my_input_dir + my_diag_suffix,decode_times=False)
dsdiag2=xr.open_dataset(my_input_dir + my_diag2_suffix,decode_times=False)


output = xr.merge([dsprog, dsdiag, dsdiag2])
output.to_netcdf(my_input_dir + my_merged_suffix)
# specify Github scratch directory where processed model output will be committed (automate later)
my_output_filename =  'aoscm.nc'
my_gitdir = '../../output_scm/AOSCM/'
# my_gitdir = 'dephy/'

## Read ModelE3 data

### Read single file containing all output data

Note: ERROR 1: PROJ... message can be ignored here.

In [42]:
input_filename = my_input_dir + my_merged_suffix


modele_data = xr.open_dataset(input_filename,decode_times=False)

# check if the run contains ice variables
do_ice = bool(max(modele_data['ice_wat_path'].values)>0.)
print('do_ice = ',do_ice)

# full parameter list

modele_data
# scm_in

do_ice =  True


<xarray.Dataset>
Dimensions:              (nlev: 137, nlevp1: 138, nlevs: 4, time: 161,
                          ntiles: 9, norg: 4, ncextr: 137)
Coordinates:
  * nlev                 (nlev) int32 1 2 3 4 5 6 7 ... 132 133 134 135 136 137
  * nlevp1               (nlevp1) int32 1 2 3 4 5 6 ... 133 134 135 136 137 138
  * nlevs                (nlevs) int32 1 2 3 4
  * time                 (time) float32 0.0 450.0 900.0 ... 7.155e+04 7.2e+04
  * ntiles               (ntiles) int32 -2147483647 -2147483647 ... -2147483647
  * norg                 (norg) int32 -2147483647 -2147483647 ... -2147483647
  * ncextr               (ncextr) int32 -2147483647 -2147483647 ... -2147483647
Data variables: (12/223)
    pressure_f           (time, nlev) float32 ...
    pressure_h           (time, nlevp1) float32 ...
    height_f             (time, nlev) float32 ...
    height_h             (time, nlevp1) float32 ...
    relative_humidity    (time, nlev) float32 ...
    t                    (time, nlev) float32 ...
    ...                   ...
    extra_col_27         (time, ncextr) float32 ...
    extra_col_28         (time, ncextr) float32 ...
    extra_col_29         (time, ncextr) float32 ...
    extra_col_30         (time, ncextr) float32 ...
    extra_col_31         (time, ncextr) float32 ...
    extra_col_32         (time, ncextr) float32 ...
Attributes:
    title:         SCM: 36r1  Sim: trref_wind
    modelID:       36r1
    simulationID:  trref_wind
    dataID:        SCM_OUTPUT
    start_day:     20200412
    start_hour:    79200

### Calculate and append additional variables

In [43]:
additional_var_list = ['surface_pressure', 'surface_friction_velocity',
                        'obukhov_length', 'inversion_height',
                        'mass_mixing_ratio_of_cloud_liquid_water_in_air',
                        'mass_mixing_ratio_of_ice_water_in_air',
                        'mass_mixing_ratio_of_rain_water_in_air',
                        'tendency_of_air_potential_temperature_due_to_radiative_heating',
                        'tendency_of_air_potential_temperature_due_to_microphysics',
                        'tendency_of_air_potential_temperature_due_to_mixing',
                        'atmosphere_mass_content_of_rain_water',
                        'air_dry_density', 
                        'surface_upwelling_longwave_flux',
                        'surface_sea_spray_number_flux',
                        'precipitation_flux_at_surface',
                        'precipitation_flux_of_ice_at_surface',
                        'vertical_flux_total_water',
                        'vertical_flux_water_vapor',
                        'zonal_momentum_flux',
                        'meridional_momentum_flux',
                        'water_vapor_mixing_ratio']


def additional_variables(var_name):

    scm_in = xr.open_dataset(my_input_dir + my_input_suffix)
    modele_data = xr.open_dataset(input_filename)
    
    Rair = 287.1
    Cp = 1003       
    kappa = 0.4     
    g = 9.8         #ms-2
    T_ice = 250.16  #K
    T_0 = 273.16    #K
    N_tot = 50      #cm-3 over ocean
    rho_liq = 1     #kgm-3
    P_0 = 100000    #Pa
    k = 2/7         
    
    ## surface pressure
    ps = scm_in.ps.data[0]*np.ones(len(modele_data.time.data))



    inv_height = np.zeros(len(modele_data.time.data))
    rain_wat_path = np.zeros(len(modele_data.time.data))

    
    ## dry air density
    rho = modele_data.pressure_f.data/(Rair*modele_data.t.data)

    ## rain water path
    for i in range(rain_wat_path.shape[0]):
        layer = modele_data.height_h.data[i,:-1] - modele_data.height_h.data[i,1:]
        rain_wat_path[i] = np.sum(modele_data.qr.data[i,:]*rho[i,:]*layer)

    ## liquid cloud fraction
    liq_cl_frct = np.zeros(modele_data.cloud_fraction.data.shape)
    for i in range(modele_data.cloud_fraction.data.shape[0]):
        ice_cld = (modele_data.t.data[i,:] <= T_ice) & (modele_data.cloud_fraction.data[i,:] > 0)
        mixed_cld = (modele_data.t.data[i,:] > T_ice) & (modele_data.t.data[i,:] < T_0) & (modele_data.cloud_fraction.data[i,:] > 0)
        warm_cld = (modele_data.t.data[i,:] > T_0) & (modele_data.cloud_fraction.data[i,:] > 0)
        liq_cl_frct[i,ice_cld] = 0
        liq_cl_frct[i,mixed_cld] = ((modele_data.t.data[i,mixed_cld] - T_ice)/(T_0 - T_ice))**2
        liq_cl_frct[i,warm_cld] = 1

    
    # upwelling longwave
    lwu = -(modele_data.sfc_lwrad.data - modele_data.sfc_lwrad_down.data)
    
    # surf. sea-spray
    sspray = np.zeros(modele_data.time.data.shape[0])
    
    # prec. fluxes
    prec = (modele_data.conv_rain.data + modele_data.stra_rain.data + 
            modele_data.conv_snow.data + modele_data.stra_snow.data )
    
    snow = modele_data.conv_snow.data + modele_data.stra_snow.data
    
    # turb. flux profiles
    turb_tot_water = (modele_data.turb_flx_wv.data + modele_data.turb_flx_liq.data + modele_data.turb_flx_ice.data)
    turb_tot_water_full = np.zeros(modele_data.t.data.shape)
    turb_tot_wv_full = np.zeros(modele_data.t.data.shape)
    for i in range(modele_data.time.data.shape[0]):
#         print(np.flipud(modele_data.height_f.data[i,:]))
        turb_tot_water_full[i,:] = -np.flipud(np.interp(np.flipud(modele_data.height_f.data[i,:]),
                                                       np.flipud(modele_data.height_h.data[i,:]),
                                                       np.flipud(turb_tot_water[i,:])))
        turb_tot_wv_full[i,:] = -np.flipud(np.interp(np.flipud(modele_data.height_f.data[i,:]),
                                                       np.flipud(modele_data.height_h.data[i,:]),
                                                       np.flipud(modele_data.turb_flx_wv.data[i,:])))

    
    
    #mixing ratios
    w = modele_data.q.data/(1-modele_data.q.data)
    w_l = modele_data.ql.data/(1-modele_data.ql.data)
    w_i = modele_data.qi.data/(1-modele_data.qi.data)
    w_r = modele_data.qr.data/(1-modele_data.qr.data)
    w_sn = modele_data.qsn.data/(1-modele_data.qsn.data)
    
    # momentum fluxes
    wu_flux = np.zeros(modele_data.t.data.shape)
    wv_flux = np.zeros(modele_data.t.data.shape)
    for i in range(modele_data.time.data.shape[0]):
        wu_flux[i,:] = -np.flipud(np.interp(np.flipud(modele_data.height_f.data[i,:]),
                                           np.flipud(modele_data.height_h.data[i,:]),
                                           np.flipud(modele_data.turb_flx_u.data[i,:])))
        wv_flux[i,:] = -np.flipud(np.interp(np.flipud(modele_data.height_f.data[i,:]),
                                           np.flipud(modele_data.height_h.data[i,:]),
                                           np.flipud(modele_data.turb_flx_v.data[i,:])))

    ## optical depths - According to IFS c43 parameterizations
    ## not computed for this experiment

#     r_e_liq = (3*modele_data.ql.data/(4*np.pi*rho_liq*0.8*N_tot))**(1/3)
#     [c_0, c_1, c_2, c_3] = [326.3, 12.42, 0.197, 0.0012]
#     r_e_ice = c_0 + c_1*T_c + c_2* T_c**2 + c_3*T_c**3
    
#     [a_0, a_1] = [0.02672, 1.32] #μmm2g-1
#     b_ext_liq = modele_data.ql.data*(a_0_liq + a_1_liq/r_e_liq)  # LWC in gm-3
#     [a_0, a_1] = [-9.45458*10**(-5), 2.52061] #μmm2g-1
#     b_ext_ice = modele_data.qi.data*(a_0_ice + a_1_ice/r_e_ice)
#     # same for rain and snow
#     b_all = b_ext_liq + b_ext_ice

#     delta = np.trapz(b_all,dp)

    ## theta tendencies
    pres_tend = np.zeros(modele_data.pressure_f.data.shape)
    
    for j in range(pres_tend.shape[1]):
        pres_tend[:,j] = np.gradient(modele_data.pressure_f.data[:,j], modele_data.time.data)
        
    theta_rad_dot = modele_data.extra_col_05.data*(P_0/modele_data.pressure_f.data)**(k) - k*modele_data.t.data*(P_0**k)*(modele_data.pressure_f.data**(-k-1))*pres_tend
    theta_turb_dot = modele_data.extra_col_08.data*(P_0/modele_data.pressure_f.data)**(k) - k*modele_data.t.data*(P_0**k)*(modele_data.pressure_f.data**(-k-1))*pres_tend
    theta_micro_dot = modele_data.extra_col_19.data*(P_0/modele_data.pressure_f.data)**(k) - k*modele_data.t.data*(P_0**k)*(modele_data.pressure_f.data**(-k-1))*pres_tend
    
    ## q tendencies
    q_turb_dot = modele_data.extra_col_09.data
    q_micro_dot = modele_data.extra_col_20.data
    
    
    ## inversion height
    t = modele_data.t.data
    t[modele_data.pressure_f.data/100<400] = np.nan
    for i in range(inv_height.shape[0]):
        t_grad = np.gradient(t[i,:],modele_data.height_f.data[i,:])
        i_grad_max = np.nonzero(t_grad==np.nanmax(t_grad))
        inv_height[i] = modele_data.height_f.data[i, i_grad_max[0]]


    ## u_star
    stress = np.sqrt(modele_data.u_sfc_strss.data**2 + modele_data.v_sfc_strss.data**2)
    u_star = np.sqrt(stress/rho[0,-1])

    ## L_star
    L_star = (u_star**3)/((kappa*g/modele_data.pot_temperature.data[:,-1])*(modele_data.sfc_sen_flx.data/(Cp*rho[:,-1])))


    if var_name == 'surface_pressure':
        return ps
    elif var_name == 'surface_friction_velocity':
        return u_star
    elif var_name == 'obukhov_length':
        return L_star
    elif var_name == 'inversion_height':
        return inv_height
    elif var_name == 'water_vapor_mixing_ratio':
        return w
    elif var_name == 'mass_mixing_ratio_of_cloud_liquid_water_in_air':
        return w_l
    elif var_name == 'mass_mixing_ratio_of_ice_water_in_air':
        return w_i + w_sn
    elif var_name == 'mass_mixing_ratio_of_rain_water_in_air':
        return w_r
    elif var_name == 'tendency_of_air_potential_temperature_due_to_radiative_heating':
        return theta_rad_dot
    elif var_name == 'tendency_of_air_potential_temperature_due_to_microphysics':
        return theta_micro_dot
    elif var_name == 'tendency_of_air_potential_temperature_due_to_mixing':
        return theta_turb_dot
    elif var_name == 'atmosphere_mass_content_of_rain_water':
        return rain_wat_path
    elif var_name == 'air_dry_density':
        return rho
    elif var_name == 'surface_upwelling_longwave_flux':
        return lwu
    elif var_name == 'surface_sea_spray_number_flux':
        return sspray
    elif var_name == 'precipitation_flux_at_surface':
        return prec
    elif var_name == 'vertical_flux_total_water':
        return turb_tot_water_full
    elif var_name == 'vertical_flux_water_vapor':
        return turb_tot_wv_full
    elif var_name == 'zonal_momentum_flux':
        return wu_flux
    elif var_name == 'meridional_momentum_flux':
        return wv_flux   


## Prepare output file in DEPHY format

### Read requested variables list

Variable description, naming, units, and dimensions.

In [44]:
# read list of requested variables
vars_mean_list = pd.read_excel('https://docs.google.com/spreadsheets/d/1Vl8jYGviet7EtXZuQiitrx4NSkV1x27aJAhxxjBb9zI/export?gid=1026157027&format=xlsx',
                              sheet_name='SCM')
pd.set_option('display.max_rows', None)
vars_mean_list

,standard_name,variable_id,units,dimensions,"comment (reported at end of each model physics time step, green=minimum, red=granularity enabling EMC2)"
0,time,time,s,–,"dimension, seconds since 2020-03-12 18:00:00"
1,pressure_layer,layer,1,–,"dimension, pressure layer number from 1 at sur..."
2,air_pressure,pa,Pa,"time, layer",pressure at mid-level points (native model lev...
3,layer_top_pressure,pe,Pa,"time, layer","dimension, pressure at layer top points (used ..."
4,surface_pressure,ps,Pa,time,–
5,surface_temperature,ts,K,time,–
6,surface_friction_velocity,ustar,m s-1,time,–
7,surface_roughness_length_for_momentum_in_air,z0,m,time,–
8,surface_roughness_length_for_heat_in_air,z0h,m,time,–
9,surface_roughness_length_for_humidity_in_air,z0q,m,time,–


### Match ModelE3 variables to requested outputs

Expand the table to include columns that indicate ModelE3 model variable names and any conversion factor.

In [45]:
# drop comments
vars_mean_list = vars_mean_list.drop(columns='comment (reported at end of each model physics time step, green=minimum, red=granularity enabling EMC2)')

# add columns to contain model output name and units conversion factors
vars_mean_list = vars_mean_list.assign(model_name='missing data',conv_factor=1.0)

In [46]:
# match to ModelE3 variable names and specify conversion factors
for index in vars_mean_list.index:
    standard_name = vars_mean_list.standard_name.iat[index]
    

    if standard_name=='surface_temperature':                                     #done
        vars_mean_list.model_name.iat[index] = 't_skin'


    if standard_name=='surface_roughness_length_for_momentum_in_air':             #done
        vars_mean_list.model_name.iat[index] = 'rough_len_mom'
    if standard_name=='surface_roughness_length_for_heat_in_air':                 #done
        vars_mean_list.model_name.iat[index] = 'rough_len_heat'
        
    if standard_name=='surface_roughness_length_for_humidity_in_air':           #to_do 
        vars_mean_list.model_name.iat[index] = 'rough_len_heat'

    if standard_name=='surface_upward_sensible_heat_flux':                        #done
        vars_mean_list.model_name.iat[index] = 'sfc_sen_flx'
        vars_mean_list.conv_factor.iat[index] = -1.
    if standard_name=='surface_upward_latent_heat_flux':                          #done
        vars_mean_list.model_name.iat[index] = 'sfc_lat_flx' 
        vars_mean_list.conv_factor.iat[index] = -1.
        
        
    if standard_name=='pbl_height':                                               #done
        vars_mean_list.model_name.iat[index] = 'pbl_height'     


    if standard_name=='atmosphere_mass_content_of_liquid_cloud_water':            #done
        vars_mean_list.model_name.iat[index] = 'liq_wat_path'                     
        vars_mean_list.conv_factor.iat[index] = 1
        

    if do_ice:
        if standard_name=='atmosphere_mass_content_of_ice_water':                  #done
            vars_mean_list.model_name.iat[index] = 'ice_wat_path'
            vars_mean_list.conv_factor.iat[index] = 1
    if standard_name=='area_fraction_cover_of_hydrometeors':                        ##done
        vars_mean_list.model_name.iat[index] = 'total_cloud'
        
        
    if standard_name=='toa_outgoing_longwave_flux':                                #done
        vars_mean_list.model_name.iat[index] = 'top_lwrad'
        vars_mean_list.conv_factor.iat[index] = -1
    if standard_name=='surface_downwelling_longwave_flux':                         #done
        vars_mean_list.model_name.iat[index] = 'sfc_lwrad_down'  
        vars_mean_list.conv_factor.iat[index] = 1

    if standard_name=='height':                                                     #done
        vars_mean_list.model_name.iat[index] = 'height_f'          

    if standard_name=='eastward_wind':                                              #done
        vars_mean_list.model_name.iat[index] = 'u'
    if standard_name=='northward_wind':                                             #done
        vars_mean_list.model_name.iat[index] = 'v'

    if standard_name=='air_temperature':                                             #done
        vars_mean_list.model_name.iat[index] = 't'

    if standard_name=='relative_humidity':                                            #done
        vars_mean_list.model_name.iat[index] = 'relative_humidity'
        vars_mean_list.conv_factor.iat[index] = 1


    if standard_name=='air_potential_temperature':                                   #done
        vars_mean_list.model_name.iat[index] = 'pot_temperature'
        
        


vars_mean_list[2:] # echo variables (first two rows are dimensions)

,standard_name,variable_id,units,dimensions,model_name,conv_factor
2,air_pressure,pa,Pa,"time, layer",missing data,1.0
3,layer_top_pressure,pe,Pa,"time, layer",missing data,1.0
4,surface_pressure,ps,Pa,time,missing data,1.0
5,surface_temperature,ts,K,time,t_skin,1.0
6,surface_friction_velocity,ustar,m s-1,time,missing data,1.0
7,surface_roughness_length_for_momentum_in_air,z0,m,time,rough_len_mom,1.0
8,surface_roughness_length_for_heat_in_air,z0h,m,time,rough_len_heat,1.0
9,surface_roughness_length_for_humidity_in_air,z0q,m,time,rough_len_heat,1.0
10,surface_upward_sensible_heat_flux,hfss,W m-2,time,sfc_sen_flx,-1.0
11,surface_upward_latent_heat_flux,hfls,W m-2,time,sfc_lat_flx,-1.0


## Create DEPHY output file

Write a single file to contain all domain-mean scalar and profile outputs. This code expects the write directory to be pre-existing (already created by the user). In the case that this output will be committed to the comble-mip GitHub repository, see above "Specify directory locations".

In [49]:
# create DEPHY output file
dephy_filename = my_gitdir + my_output_suffix
if os.path.exists(dephy_filename):
    os.remove(dephy_filename)
    print('The file ' + dephy_filename + ' has been deleted successfully')    
dephy_file = Dataset(dephy_filename,mode='w',format='NETCDF3_CLASSIC')
start_date = '2020-03-12T22:00:00Z'

# create global attributes
dephy_file.title='AOSCM results for COMBLE-MIP case: fixed stratiform Nd and Ni'
dephy_file.reference='https://github.com/ARM-Development/comble-mip'
dephy_file.authors='Michail Karalis (michail.karalis@misu.su.se), Gunilla Svensson (gunilla@misu.su.se)'
dephy_file.source=input_filename
dephy_file.version=dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dephy_file.format_version='DEPHY SCM format version 1.6'
dephy_file.script='convert_AOSCM_output_to_dephy_format.ipynb'
dephy_file.startDate=start_date
dephy_file.force_geo=1
dephy_file.surfaceType='ocean'
dephy_file.surfaceForcing='ts'
dephy_file.lat='74.5 deg N'
dephy_file.dp='see pressure variable'
dephy_file.np=modele_data.sizes['nlev']

# create dimensions
nt = modele_data.dims['time']
time = dephy_file.createDimension('time', nt)
time = dephy_file.createVariable('time', np.float64, ('time',))
time.units = 'seconds since ' + dephy_file.startDate
time.long_name = 'time'
# find time step and build time in seconds
# time1 = dt.datetime.strptime(str(modele_data['time'].data[0]),'%Y-%m-%d %H:%M:%S')
# time2 = dt.datetime.strptime(str(modele_data['time'].data[1]),'%Y-%m-%d %H:%M:%S')
# delta_t = (time2-time1).total_seconds()


delta_t = modele_data['time'].data[1]
# time[:] = (np.arange(nt)+1.)*delta_t
time[:] = modele_data['time'].data

nl = modele_data.dims['nlev']
# pa = dephy_file.createDimension('pressure', nl)
# pa = dephy_file.createVariable('pressure', np.float64, ('pressure',))
# pa.units = 'Pa'
# pa.long_name = 'pressure'
# pa[:] = modele_data['pressure_f'].data[0,:]*100.

lvl = dephy_file.createDimension('pa', nl)
pa = dephy_file.createVariable('pa', np.float64, ('pa',))
pa.units = 'Pa'
pa.long_name = 'pressure'
pa[:] = modele_data['pressure_f'].data[0,:]


# create and fill variables
for index in vars_mean_list.index[3:]:
    std_name = vars_mean_list.standard_name.iat[index]
    print(std_name)
#   print(std_name) # debug
    var_name = vars_mean_list.variable_id.iat[index]
    mod_name = vars_mean_list.model_name.iat[index]
    c_factor = vars_mean_list.conv_factor.iat[index]
    if vars_mean_list.dimensions.iat[index]=='time':
        new_sca = dephy_file.createVariable(var_name, np.float64, ('time'))
        new_sca.units = vars_mean_list.units.iat[index]
        new_sca.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data':
            new_sca[:] = modele_data[mod_name].data*c_factor
        elif np.any(np.array(additional_var_list) == std_name) :
            new_sca[:] = additional_variables(std_name)
    if vars_mean_list.dimensions.iat[index]=='time, layer':
        new_snd = dephy_file.createVariable(var_name, np.float64, ('time','pa'))
        new_snd.units = vars_mean_list.units.iat[index]
        new_snd.long_name = std_name
        if vars_mean_list.model_name.iat[index]!='missing data': 
            new_snd[:] = modele_data[mod_name].data*c_factor
        elif np.any(np.array(additional_var_list) == std_name) :
            new_snd[:] = additional_variables(std_name)
    


dephy_file.close()

layer_top_pressure
surface_pressure
surface_temperature
surface_friction_velocity
surface_roughness_length_for_momentum_in_air
surface_roughness_length_for_heat_in_air
surface_roughness_length_for_humidity_in_air
surface_upward_sensible_heat_flux
surface_upward_latent_heat_flux
obukhov_length
pbl_height
inversion_height
atmosphere_mass_content_of_liquid_cloud_water
atmosphere_mass_content_of_rain_water
atmosphere_mass_content_of_ice_water
area_fraction_cover_of_hydrometeors
area_fraction_cover_of_liquid_cloud
area_fraction_cover_of_convective_hydrometeors
optical_depth
optical_depth_of_liquid_cloud
precipitation_flux_at_surface
precipitation_flux_of_ice_at_surface
toa_outgoing_longwave_flux
surface_downwelling_longwave_flux
surface_upwelling_longwave_flux
surface_sea_spray_number_flux
height
eastward_wind
northward_wind
air_dry_density
air_temperature
water_vapor_mixing_ratio
relative_humidity
relative_humidity_over_ice
air_potential_temperature
mass_mixing_ratio_of_cloud_liquid_water_

### Check output file

In [9]:
dephy_check = xr.open_dataset(dephy_filename)
dephy_check

<xarray.Dataset>
Dimensions:    (time: 40, pa: 110)
Coordinates:
  * time       (time) datetime64[ns] 2020-03-12T22:30:00 ... 2020-03-13T18:00:00
  * pa         (pa) float64 9.79e+04 9.69e+04 9.59e+04 ... 1.4 0.75 0.35
Data variables: (12/94)
    ps         (time) float64 ...
    ts         (time) float64 ...
    ustar      (time) float64 ...
    z0         (time) float64 ...
    z0h        (time) float64 ...
    z0q        (time) float64 ...
    ...         ...
    vmics      (time, pa) float64 ...
    vmips      (time, pa) float64 ...
    vmlcc      (time, pa) float64 ...
    vmlrc      (time, pa) float64 ...
    vmicc      (time, pa) float64 ...
    vmipc      (time, pa) float64 ...
Attributes: (12/14)
    title:           ModelE3 SCM results for COMBLE-MIP case: fixed stratifor...
    reference:       https://github.com/ARM-Development/comble-mip
    authors:         Ann Fridlind (ann.fridlind@nasa.gov), Florian Tornow (fl...
    source:          /data/home/fridlind/modelE3//allsteps.allmergeSCM_COMBLE...
    version:         2023-11-30 00:16:50
    format_version:  DEPHY SCM format version 1.6
    ...              ...
    force_geo:       1
    surfaceType:     ocean
    surfaceForcing:  ts
    lat:             74.5 deg N
    dp:              see pressure variable
    np:              110